# Proyecto de análisis de sentimientos con Python

##### Carmen Giles Floriano

En este proyecto se realizará un modelo de aprendizaje automático capaz de analizar tweets y predecir el sentimiento del usuario dentro de las siguientes categorías: "Muy feliz", "Contento", "Neutro", "Molesto" y "Hater". 

#### Ejercicio 1. RECOPILACIÓN DE DATOS

In [22]:
import pandas 
import numpy
import re
import nltk
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from textblob import TextBlob


En primer lugar, se lee el fichero `fifa_tweets_emotion.csv` mediante el paquete pandas. Los datos fueron obtenidos de Kaggle y son un conjunto de tweets relacionados con la WorldCup del FIFA 2022. 

In [2]:
# Con la función "read_csv" del paquete "pandas", leemos el data set indicando los nombres de las columnas.
tweet_fifa_prev = pandas.read_csv('fifa_tweets_emotion.csv', header=None,
                       names=['number', 'date', 'number_likes','source', 'tweet','feeling'])
print(tweet_fifa_prev.shape)
tweet_fifa_prev.head(5)

(22525, 6)


,number,date,number_likes,source,tweet,feeling
0,NaN,Date Created,Number of Likes,Source of Tweet,Tweet,Sentiment
1,0.0,2022-11-20 23:59:21+00:00,4,Twitter Web App,What are we drinking today @TucanTribe \n@MadB...,neutral
2,1.0,2022-11-20 23:59:01+00:00,3,Twitter for iPhone,Amazing @CanadaSoccerEN #WorldCup2022 launch ...,positive
3,2.0,2022-11-20 23:58:41+00:00,1,Twitter for iPhone,Worth reading while watching #WorldCup2022 htt...,positive
4,3.0,2022-11-20 23:58:33+00:00,1,Twitter Web App,Golden Maknae shinning bright\n\nhttps://t.co/...,positive


El data set presenta 22525 tweets distintos, cada uno representado en una fila. Además, del texto del propio tweet existen otras columnas con más información, como la fecha de publicación, el número de likes de la publicación, el lugar desde dónde se publicó y el sentimiento asociado. 

En este proyecto, sólo nos interesa el texto correspondiente al tweet. Los demás datos son eliminados, incluso la columna correspondiente a los sentimientos, ya que estos serán asignados siguiendo el criterio de la herramienta TextBlob. 

In [3]:
# Con la función "drop" se eliminan elementos del data set.
## En la lista "delete_col" añadimos los nombres de las columnas que deseamos eliminar, la cuál será un parámetro de la función
## "drop". Además, a esta función se le da el parámetro axis=1, lo que indica que los elementos a eliminar son columnas.

delete_col=["number", "date", "number_likes", "source", "feeling"]
tweet_fifa_col= tweet_fifa_prev.drop(delete_col, axis=1)

## Por otro lado, eliminamos la primera fila que contiene los anteriores nombres de las columnas del data set. Se elimina 
## indicando 0 (primer elemento) y el parámetro axis=0 (fila). 

tweet_fifa=tweet_fifa_col.drop(0, axis=0)
tweet_fifa.head(5)


,tweet
1,What are we drinking today @TucanTribe \n@MadB...
2,Amazing @CanadaSoccerEN #WorldCup2022 launch ...
3,Worth reading while watching #WorldCup2022 htt...
4,Golden Maknae shinning bright\n\nhttps://t.co/...
5,"If the BBC cares so much about human rights, h..."


#### Ejercicio 2. LIMPIEZA DEL TEXTO, ELIMINAR LAS PALABRAS QUE NO APORTAN INFORMACIÓN.

A continuación, procesamos el texto del tweet de forma que sea más fácil de codificar y analizar en pasos posteriores. Para ello se crea la función "limpiar_texto", que elimina elementos como menciones, hashtags, URLs y emoticonos; convierte el texto en minúscula; elimina las palabras poco informativas; y, además, realizará una lematización, es decir, transforma las palabras a su forma base.

In [14]:
## Recibe como entrada un conjunto de datos (data set).
def limpiar_texto(data_set):
    
    ## En primer lugar, se cargan una serie de datos necesarios para el posterior procesamiento:
    
    # Se cargan los datos que hacen referencia a distintos emoticonos, utilizando "re", un módulo de la biblioteca de Pyhton. 
    # En concreto, con la funcion "compile" se almacena dicha información indicando el código Unicode del bloque de emoticonos
    # correspondiente. El parámetro "flags=re.UNICODE" indica que la información añadida se lea según las reglas de Unicode, 
    # para que sean identificados como emoticonos.
    patron_emoticonos = re.compile("["
                            u"\U0001F600-\U0001F64F"  # Emoticonos generales
                            u"\U0001F300-\U0001F5FF"  # Símbolos y pictogramas
                            u"\U0001F680-\U0001F6FF"  # Transporte y mapas
                            u"\U0001F780-\U0001F7FF"  # Formas geométricas extendidas
                            u"\U0001F800-\U0001F8FF"  # Signos de puntuación
                            u"\U00002702-\U000027B0"  # Símbolos diversos, como tijeras o meteorológicos
                            u"\U000024C2-\U0001F251"  # Transporte, pictogramas y otros símbolos
                            u"\U0001F900-\U0001F9FF"  # Emoticonos de personas y cuerpos
                            u"\U0001FA00-\U0001FA6F"  # Símbolos de objetos
                            u"\U0001FA70-\U0001FAFF"  # Símbolos de alimentos
                            u"\U0001F1E6-\U0001F1FF"  # Banderas 
                            "]+", flags=re.UNICODE)
    
    # Se carga la información de la función "TweetTokenize" del paquete "nltk". 
    tokenizer=TweetTokenizer()
    
    # Se carga el conjunto de palabras "stopwords" del paquete nltk, estas son palabras muy frecuentes en el lenguaje pero poco
    # informativas. 
    nltk.download("stopwords")
    stopwords_english = stopwords.words("english") # Se selecciona el idioma inglés.
    
    # Se carga la información necesaria para la lematización mediante la función "SnowballStemmer" del paquete "nltk".
    stemmer = SnowballStemmer("english")
    
    ## Se inicia un diccionario que alamacenará los tweets procesados.
    diccionario = {"tweet": []}
    
    ## Para cada fila del data set: 
    for index, fila in data_set.iterrows():
        
        ## En la variable fila se tiene la información de la fila completa, al seleccionar "tweet", nos quedamos sólo con la 
        ## información de esa columna, es decir, el texto del tweet correspondiente.
        tweet=fila["tweet"]
        
        ## Con la función "sub" del módulo "re" se eliminan una serie de elementos, indicados en el primer parámetro de la 
        ## función (r'elemento'). 
        
        # Se eliminan los hashtags (#)
        tweet = re.sub(r'#', '', tweet)
        
        # Se eliminan las menciones, las cuáles tienen la siguiente estructura: @usuario. Al añadir \S+, se indica que también 
        # se elimina cualquier texto que esté inmediatamente después del elemento determinado, sin incluir espacios.
        tweet = re.sub(r'@\S+', '', tweet)
        
        # Se eliminan los URLs, que por lo general comienzan con http.
        tweet = re.sub(r'http\S+', '', tweet) 
        
        # Se eliminan los emoticonos usando el conjunto de emoticonos anteriormente guardados.
        tweet = patron_emoticonos.sub(r'',tweet)
        
        ## Se transforma todo el texto a minúscula.
        tweet=tweet.lower()
        
        ## Posteriormente, se procesará el texto palabra por palabra. Para ello, en primer lugar, se debe tokenizar el texto, 
        ## es decir, separar las palabras a elementos de una lista. Se realiza mediante la función "TweetTokenizer" cargada 
        ## anteriormente.
        tweet= tokenizer.tokenize(tweet)
        ## Se inicia una lista que contendrá las palabras procesadas del tweet.
        list_tweet=[]

        ## Para cada palabra:
        for palabra in tweet:
            ## Si la palabra NO está en el conjunto "stopwords_english", palabras con poco valor informativo:
            if palabra not in stopwords_english:
                ## Se transforma a su forma base con la función "SnowballStemmer".
                palabra_proc = stemmer.stem(palabra)
                ## La palabra ya procesada se añade a la lista anteriormente creada.
                list_tweet.append(palabra_proc)
        
        ## Una vez procesadas las palabras del tweet, se vuelven a unir en la cadena que se inicia a continuación.
        cadena_tweet=""
        
        ## Para cada palabra:
        for palabra in list_tweet:
            ## Se añade cada palabra a la cadena anteriormente creada, separadas por espacios.
            cadena_tweet+=" "
            cadena_tweet+=palabra
        
        ## El tweet procesado se añade al diccionario    
        diccionario["tweet"].append(cadena_tweet)
    
    ## Se convierte el diccionario en un data set, mediante el paquete "pandas", y se devuelve.
    data_set_proc=pandas.DataFrame(diccionario)
    return data_set_proc

In [15]:
tweet_fifa_limp=limpiar_texto(tweet_fifa)
tweet_fifa_limp.head(5)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,tweet
0,drink today worldcup 2022
1,amaz worldcup 2022 launch video . show much f...
2,worth read watch worldcup 2022
3,golden makna shin bright jeonjungkook jungkoo...
4,"bbc care much human right , homosexu right , ..."


Finalmente, se obtienen un data set con tweets libres de emoticonos, de palabras pocas informativas y de otros elementos que no interesan a la hora de analizar el sentimiento de un tweet.

#### Ejercicio 3. ETIQUETADO DE DATOS CON HERRAMIENTAS YA EXISTENTES

A continuación, se creará una función denominada "clasificador" con la cuál se asignará un sentimiento a cada tweet. Esto se realizará con el modelo "TextBlob" ya existente. Este asigna una puntuación (polaridad) a un texto dado, desde -1 a 1. Una puntuación cercana a -1 hace referencia a un texto con un sentimiento muy negativo y, cercana a 1, muy positivo. De esta forma, se asignan los sentimientos de la siguiente forma:

Muy feliz: polaridad=[-1,-0.6)

Contento: polaridad=[-0.6, -0.2)

Neutro: polaridad=[-0.2, 0.2)

Molesto: polaridad=[0.2, 0.6)

Hater: polaridad=[0.6, 1]

In [16]:
## Como entrada la función recibe un data set.
def clasificador(data_set):
    
    ## Se inicia un diccionario vacio, para guardar, por un lado, el tweet y, por otro lado, el sentimiento asociado.
    diccionario={"tweet":[], "sentimiento":[]}
    
    ## Para cada fila del data set:
    for index, fila in data_set.iterrows():
        
        ## Se extrae el texto del tweet correspondiente.
        tweet=fila["tweet"]
        
        ## Se analiza con el paquete TextBlob.
        texto=TextBlob(tweet)
        
        ## Se extrae el valor de polaridad que, como se explicó anteriormente, está asociado a un sentimiento. Siguiendo el 
        ## criterio anteriormente marcado, dependiendo del valor de polaridad calculado se asigna un sentimiento.
        sentimiento_pol=texto.sentiment.polarity
        if sentimiento_pol < (-0.6):
            sentimiento="Hater"
        elif (-0.6) <= sentimiento_pol < (-0.2):
            sentimiento="Molesto"
        elif (-0.2) <= sentimiento_pol < 0.2:
            sentimiento="Neutro"
        elif 0.2 <= sentimiento_pol < 0.6:
            sentimiento="Contento"
        else:
            sentimiento="Muy feliz"
        
        ## El tweet se almacena en el diccionario.
        diccionario["tweet"].append(tweet)
        
        ## El sentimiento, guardado en una cadena de carácteres, se almacena en el diccionario, generando una lista de cadenas.
        diccionario["sentimiento"].append(sentimiento)
    
    ## El diccionario se convierte en un data set y se devuelve.
    data_set_sent=pandas.DataFrame(diccionario)
    return data_set_sent

In [17]:
tweet_fifa_sent=clasificador(tweet_fifa_limp)
tweet_fifa_sent.head(5)

,tweet,sentimiento
0,drink today worldcup 2022,Neutro
1,amaz worldcup 2022 launch video . show much f...,Neutro
2,worth read watch worldcup 2022,Contento
3,golden makna shin bright jeonjungkook jungkoo...,Contento
4,"bbc care much human right , homosexu right , ...",Neutro


De esta forma, se obtiene un data set con dos columnas, una con los tweets limpios y otra con el sentimiento asociado a dicho tweet. Este conjunto de datos será codificado y usado para entrenar un modelo propio, que permita asignar dichos sentimientos a nuevos tweets de prueba.

#### Ejercicio 4. CODIFICACIÓN DE LOS ATRIBUTOS Y OBJETIVOS.

In [19]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\USUARIO\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

El siguiente paso para crear un modelo, es codificar los tweets a lenguaje numérico. Existen múltiples codificadores con distintas características. Por ejemplo, se tiene el codificador **CountVectorizer**, el cuál se basa en realizar recuentos de las distintas palabras, sin tener en cuenta la semántica o la posición de estas. Por esa razón, se buscó otras opciones más apropiadas para analizar tweets, dónde el contexto de las palabras es importante. Se estudiará el codificador **Word2Vec**, el cuál sí que tiene en cuenta la relación entre las palabras, esto es útil, por ejemplo, para poder procesar expresiones o frases hechas. Este modelo asignará a cada palabra un vector con unas dimensiones determinadas. El modelo se basa en que las palabras que se parecen estarán representadas por vectores cercanos en el espacio. A continuación, vemos como se realiza una codificación de este tipo.

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize
nltk.download('punkt')

In [31]:
#### CODIFICADOR WORD2VEC

## Se obtiene la lista de tweets procesados.
tweets=tweet_fifa_sent["tweet"]

## Este codificador necesita que el texto este tokenizado, es decir, que cada palabra sea un elemento de una lista. Esto se 
## aplica para cada tweet, mediante la función "TweetTokenizer" del paquete "nltk" y se almacena en una lista.
tokens_list=[]
tokenizer=TweetTokenizer()
for tweet in tweets:
    tokens= tokenizer.tokenize(tweet)
    tokens_list.append(tokens)


## Se entrena el modelo de codificación Word2Vec, utilizando la lista de tokens previamente generada. El parámetro 
## "vector_size=100" indica que el vector que representa cada palabra tiene 100 dimensiones; por otro lado, windows=7 
## indica que considera 7 palabras a cada lado de la analizada para entender el contexto; y min_count=5 indica que dicha 
## palabra aparecer mínimo una vez para aparecer en el modelo. 
model = Word2Vec(tokens_list, vector_size=100, window=7, min_count=5)

## Se inicia un diccionario que contendrá la codificación para cada tweet.
atributo={"tweet_vector":[]}

## Para cada tweet tokenizado:
for tweet in tokens_list:
    lista=[]
    ## Para cada palabra de dicho tweet:
    for palabra in tweet:
        ## Si la palabra está presente en el modelo previamente creado:
        if palabra in model.wv:
            ## Se realiza la codificación, o vector, de dicha palabra y se añade a la lista anteriormente creada.
            codificacion = model.wv[palabra]
            lista.append(codificacion)
    ## Si no está la lista vacia:
    if lista:
        ## Se calcula el promedio de todos los vectores para tener una representación única del tweet.
        media_vector=sum(lista)/len(lista)
        ## Dicha media se añade al diccionario "atributos".
        atributo["tweet_vector"].append(media_vector)
    ## Si está vacia:
    else:
        ## Se añade un elemento vacio a la lista.
        atributo["tweet_vector"].append([])

##         
atributo=pandas.DataFrame(atributo)

objetivo=tweet_fifa_sent["sentimiento"]

(atributos_entrenamiento, atributos_prueba,
 objetivo_entrenamiento, objetivo_prueba) = train_test_split(
       atributo, objetivo,
       random_state=12345,
       test_size=.2,
       stratify=objetivo)


In [24]:
atributos_1=tweet_fifa_sent["tweet"]
vectorizer = CountVectorizer()
atributos_1 = vectorizer.fit_transform(atributos_1)

#Extrae el objetivo (ya es numérico)
objetivo_1= tweet_fifa_sent["sentimiento"]
#Separa conjunto de entrenamiento y de prueba
(atributos_entrenamiento_1, atributos_prueba_1,
 objetivo_entrenamiento_1, objetivo_prueba_1) = train_test_split(
        atributos_1, objetivo_1,
        random_state=12345,
        test_size=.2,
        stratify=objetivo)


In [32]:
#### RANDOM FOREST
from sklearn.ensemble import RandomForestClassifier

# Entrenar el modelo Random Forest
rf_classifier = RandomForestClassifier()
rf_classifier.fit(list(atributos_entrenamiento["tweet_vector"]), objetivo_entrenamiento)

# Realizar predicciones con el conjunto de prueba
predicciones_rf = rf_classifier.predict(list(atributos_prueba["tweet_vector"]))

# Calcular la precisión del modelo
precision_rf = numpy.mean(predicciones_rf == objetivo_prueba)
print("La precisión del modelo Random Forest es", precision_rf * 100)

La precisión del modelo Random Forest es 71.8312985571587


In [25]:
#### NAIVE BAYES
from sklearn.naive_bayes import MultinomialNB

#Entrena el modelo de Naive Bayes usando la instancia MultinomialNB que es recomendada 
#para este tipo de tareas
emotion_detector = MultinomialNB(alpha=1.0)  # alpha es el parámetro de suavizado
emotion_detector.fit(atributos_entrenamiento_1, objetivo_entrenamiento_1)

#Realiza las predicciones con el conjunto de prueba
predicciones = emotion_detector.predict(atributos_prueba_1)
#Calcular la precisión del modelo
precision = emotion_detector.score(atributos_prueba_1, objetivo_prueba_1)
print("La precisión del modelo Naive Bayes desarrollado es", precision*100)

La precisión del modelo Naive Bayes desarrollado es 71.03218645948945
